<a href="https://colab.research.google.com/github/krawch1989/iowa_liquor_sales/blob/stage/main_iowa_liquor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from google.colab import auth
auth.authenticate_user()
from google.cloud import bigquery

In [0]:
eth_project_id = 'earnest-star-194115'
client = bigquery.Client(project=eth_project_id)

In [0]:
sql = """
    SELECT *
    FROM `bigquery-public-data.iowa_liquor_sales.sales`
    where date BETWEEN DATE('2019-01-01') AND DATE('2019-12-31')
"""

df = client.query(sql).to_dataframe()

In [0]:
df.head()

In [0]:
df.dtypes

In [0]:
df.date = pd.to_datetime(df.date)
df['dayofweek'] = df.date.dt.weekday_name
df['month'] = df.date.dt.month_name()
df.loc[df.month.isin(['December','January','February']),'season'] = 'winter'
df.loc[df.month.isin(['March','April','May']),'season'] = 'spring'
df.loc[df.month.isin(['June','July','August']),'season'] = 'summer'
df.loc[df.month.isin(['September','October','November']),'season'] = 'autumn'

In [0]:
df.store_location = df.store_location.str.replace('POINT \(', '').str.replace(')', '').str.strip().str.split(' ')
len(df.loc[df.store_location.isnull()])/len(df)

In [0]:
df.loc[df.store_location.notnull(), 'longtitude'] = df.loc[df.store_location.notnull(), 'store_location'].apply(lambda x: x[0]).astype(float).round(4)
df.loc[df.store_location.notnull(), 'latitude'] = df.loc[df.store_location.notnull(), 'store_location'].apply(lambda x: x[1]).astype(float).round(4)

In [0]:
df.vendor_number.value_counts()

In [0]:
df.vendor_name.value_counts()

In [0]:
cat_number = ['store_number','county_number','category','vendor_number'] 
cat_name = ['store_name','county','category_name','vendor_name']
for number, name in zip(cat_number, cat_name):
  print(cat_number,cat_name)
  frame = df[[cat_number, cat_name]].drop_duplicates(subset = cat_number)
  del df[cat_name]
  df = df.merge(frame, how='left', on=cat_number)
  del df[cat_number]
del frame

In [0]:
df.head()

In [0]:
df.drop(columns=['address', 'store_location'])

In [23]:
df.store_name.value_counts()

Hy-Vee #3 / BDI / Des Moines           21787
Central City 2                         17816
Central City Liquor, Inc.              15706
Hy-Vee Food Store / Cedar Falls        15044
Hy-Vee Wine and Spirits / Iowa City    13503
                                       ...  
Katy Did's General Store                   9
Hometown Foods - Conrad                    9
Templeton Distilling LLC                   4
Iowa Legendary Rye                         4
Paradise Distilling Company                2
Name: store_name, Length: 1759, dtype: int64

In [0]:
df.drop(columns=[])

invoice_and_item_number     object
date                        object
store_number                object
store_name                  object
address                     object
city                        object
zip_code                    object
store_location              object
county_number               object
county                      object
category                    object
category_name               object
vendor_number               object
vendor_name                 object
item_number                 object
item_description            object
pack                         int64
bottle_volume_ml             int64
state_bottle_cost          float64
state_bottle_retail        float64
bottles_sold                 int64
sale_dollars               float64
volume_sold_liters         float64
volume_sold_gallons        float64
dtype: object

In [11]:
df.shape

(2380345, 24)

In [12]:
df.shape

(2380345, 24)

invoice_and_item_number	STRING	NULLABLE	Concatenated invoice and line number associated with the liquor order. This provides a unique identifier for the individual liquor products included in the store order

date	DATE	NULLABLE	Date of order

store_number	STRING	NULLABLE	Unique number assigned to the store who ordered the liquor.

store_name	STRING	NULLABLE	Name of store who ordered the liquor.

address	STRING	NULLABLE	Address of store who ordered the liquor.

city	STRING	NULLABLE	City where the store who ordered the liquor is located

zip_code	STRING	NULLABLE	Zip code where the store who ordered the liquor is located

store_location	STRING	NULLABLE	Location of store who ordered the liquor. The Address, City, State and Zip Code are geocoded to provide geographic coordinates. Accuracy of geocoding is dependent on how well the address is interpreted and the completeness of the reference data used.

county_number	STRING	NULLABLE	Iowa county number for the county where store who ordered the liquor is located

county	STRING	NULLABLE	County where the store who ordered the liquor is located

category	STRING	NULLABLE	Category code associated with the liquor ordered

category_name	STRING	NULLABLE	Category of the liquor ordered.

vendor_number	STRING	NULLABLE	The vendor number of the company for the brand of liquor ordered

vendor_name	STRING	NULLABLE	The vendor name of the company for the brand of liquor ordered

item_number	STRING	NULLABLE	Item number for the individual liquor product ordered.

item_description	STRING	NULLABLE	Description of the individual liquor product ordered.

pack	INTEGER	NULLABLE	The number of bottles in a case for the liquor ordered

bottle_volume_ml	INTEGER	NULLABLE	Volume of each liquor bottle ordered in milliliters.

state_bottle_cost	FLOAT	NULLABLE	The amount that Alcoholic Beverages Division paid for each bottle of liquor ordered

state_bottle_retail	FLOAT	NULLABLE	The amount the store paid for each bottle of liquor ordered

bottles_sold	INTEGER	NULLABLE	The number of bottles of liquor ordered by the store

sale_dollars	FLOAT	NULLABLE	Total cost of liquor order (number of bottles multiplied by the state bottle retail)

volume_sold_liters	FLOAT	NULLABLE	Total volume of liquor ordered in liters. (i.e. (Bottle Volume (ml) x Bottles Sold)/1,000)

volume_sold_gallons	FLOAT	NULLABLE	Total volume of liquor ordered in gallons. (i.e. (Bottle Volume (ml) x Bottles Sold)/3785.411784)